In [1]:
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
mask = np.load(r"C:\Users\DU\hybrid_cascade\fastmri\Data\mask_4x_320_random.npy")  # Shape: (1, 320, 320)

print("og_mask shape:", mask.shape)

og_mask shape: (1, 1, 320, 1)


In [3]:
import h5py
import numpy as np
import tensorflow as tf

class MRISliceGenerator(tf.keras.utils.Sequence):
    """
    Generator for U-Net MRI reconstruction.
    Input : undersampled image  (B, H, W, 2)
    Target: fully sampled image (B, H, W, 2)
    """

    def __init__(self, file_list, batch_size=4, shuffle=True):
        self.file_list = file_list
        self.batch_size = batch_size
        self.shuffle = shuffle

        self.slice_index_map = []
        self._build_index()

    def _build_index(self):
        """Build (file_idx, slice_idx) mapping"""
        for file_idx, file_path in enumerate(self.file_list):
            with h5py.File(file_path, 'r') as f:
                num_slices = f['image_under'].shape[0]
                for slice_idx in range(num_slices):
                    self.slice_index_map.append((file_idx, slice_idx))
        self.on_epoch_end()

    def __len__(self):
        """Number of batches per epoch"""
        return int(np.ceil(len(self.slice_index_map) / self.batch_size))

    def __getitem__(self, index):
        batch_map = self.slice_index_map[
            index * self.batch_size : (index + 1) * self.batch_size
        ]

        x_batch = []
        y_batch = []

        for file_idx, slice_idx in batch_map:
            with h5py.File(self.file_list[file_idx], 'r') as f:
                # undersampled image (H, W, 2)
                x = f['image_under'][slice_idx]
                # fully sampled GT (H, W, 2)
                y = f['image_full'][slice_idx]

                x_batch.append(x)
                y_batch.append(y)

        x_batch = np.stack(x_batch, axis=0).astype(np.float32)
        y_batch = np.stack(y_batch, axis=0).astype(np.float32)

        return x_batch, y_batch

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.slice_index_map)


In [4]:
train_folder = r"D:\fastmri_singlecoil_FSSCAN\train_sub_norm"
val_folder = r"D:\fastmri_singlecoil_FSSCAN\val_norm"

In [5]:
import h5py
import numpy as np
import glob
import os
kspace_files_list_train = sorted(glob.glob(os.path.join(train_folder, "*.h5")))
kspace_files_list_val = sorted(glob.glob(os.path.join(val_folder, "*.h5")))

# half_train = 20
# half_val = 10
half_train = len(kspace_files_list_train) 
half_val = len(kspace_files_list_val) 
# print("half_train",half_train)
# print("half_val",half_val)
kspace_files_list_train = kspace_files_list_train[:]
kspace_files_list_val = kspace_files_list_val[:]

# Create generators
# train_gen = MRISliceGenerator(kspace_files_list_train,batch_size=16, shuffle=True,mask=mask)
# val_gen = MRISliceGenerator(kspace_files_list_val, batch_size=4, shuffle=False,mask=mask)
train_gen = MRISliceGenerator(kspace_files_list_train,batch_size=8, shuffle=True)
val_gen = MRISliceGenerator(kspace_files_list_val, batch_size=4, shuffle=False)

print(len(train_gen))  
print(len(val_gen))  


1086
1784


In [14]:
from skimage.metrics import peak_signal_noise_ratio
import os
import h5py
import numpy as np
from tqdm import tqdm


def nmse(gt: np.ndarray, pred: np.ndarray) -> np.ndarray:
    """Compute Normalized Mean Squared Error (NMSE)"""
    return np.array(np.linalg.norm(gt - pred) ** 2 / np.linalg.norm(gt) ** 2)

# Your preprocessed folder
save_dir = r'D:\fastmri_singlecoil_FSSCAN\val_norm'
files = sorted([os.path.join(save_dir, f) for f in os.listdir(save_dir) if f.endswith('.h5')])

nmse_all_volumes = []

for file in tqdm(files, desc="Computing PSNR and NMSE"):
    with h5py.File(file, 'r') as f:
        image_full = f['image_full'][:]   # shape: [slices, H, W, 2]
        image_under = f['image_under'][:]
        # max_val = f['max_val'][()]  # Optional, if stored

    # Convert to complex
    image_full_complex = image_full[..., 0] + 1j * image_full[..., 1]
    image_under_complex = image_under[..., 0] + 1j * image_under[..., 1]

    # Convert to magnitude
    mag_gt = np.abs(image_full_complex)
    mag_under = np.abs(image_under_complex)

    # Compute PSNR and NMSE
    #volume_psnr = psnr(mag_gt, mag_under)
        # PSNR slice-wise
    nmses = [nmse(mag_gt[i], mag_under[i]) for i in range(mag_gt.shape[0])]
    mean_nmse = np.mean(nmses)

    nmse_all_volumes.append(mean_nmse)
    print(f"{os.path.basename(file)}: NMSE = {mean_nmse:.2f} dB")

# Overall statistics
# avg_psnr = np.mean(psnr_all_volumes)
# std_psnr = np.std(psnr_all_volumes)
avg_nmse = np.mean(nmse_all_volumes)
std_nmse = np.std(nmse_all_volumes)

print("\n" + "="*40)
#print(f"Overall Average PSNR: {avg_psnr:.2f} ± {std_psnr:.2f} dB")
print(f"Overall Average NMSE: {avg_nmse:.6f} ± {std_nmse:.6f}")
print("="*40)


Computing PSNR and NMSE:   2%|▊                                                        | 3/199 [00:00<00:21,  9.18it/s]

volume_000.h5: NMSE = 0.06 dB
volume_001.h5: NMSE = 0.02 dB
volume_002.h5: NMSE = 0.03 dB


Computing PSNR and NMSE:   3%|█▍                                                       | 5/199 [00:00<00:22,  8.74it/s]

volume_003.h5: NMSE = 0.03 dB
volume_004.h5: NMSE = 0.04 dB
volume_005.h5: NMSE = 0.06 dB


Computing PSNR and NMSE:   4%|██▎                                                      | 8/199 [00:00<00:19,  9.97it/s]

...

volume_PDFS_092.h5: NMSE = 0.06 dB
volume_PDFS_093.h5: NMSE = 0.08 dB
volume_PDFS_094.h5: NMSE = 0.09 dB


Computing PSNR and NMSE:  99%|██████████████████████████████████████████████████████▍| 197/199 [00:20<00:00,  9.59it/s]

volume_PDFS_095.h5: NMSE = 0.06 dB
volume_PDFS_096.h5: NMSE = 0.05 dB


Computing PSNR and NMSE: 100%|███████████████████████████████████████████████████████| 199/199 [00:20<00:00,  9.67it/s]

volume_PDFS_097.h5: NMSE = 0.04 dB
volume_PDFS_098.h5: NMSE = 0.05 dB

Overall Average NMSE: 0.055463 ± 0.025401


In [16]:
import os
import h5py
import numpy as np


def nmse(gt: np.ndarray, pred: np.ndarray) -> float:
    """Compute Normalized Mean Squared Error (NMSE)"""
    return np.linalg.norm(gt - pred) ** 2 / np.linalg.norm(gt) ** 2


# Path to preprocessed validation folder
save_dir = r'D:\fastmri_singlecoil_FSSCAN\val_norm'
files = sorted([
    os.path.join(save_dir, f)
    for f in os.listdir(save_dir)
    if f.endswith('.h5')
])

nmse_all_volumes = []

for file in files:
    with h5py.File(file, 'r') as f:
        image_full = f['image_full'][:]   # (slices, H, W, 2)
        image_under = f['image_under'][:]

    # Convert to complex
    image_full_complex = image_full[..., 0] + 1j * image_full[..., 1]
    image_under_complex = image_under[..., 0] + 1j * image_under[..., 1]

    # Convert to magnitude
    mag_gt = np.abs(image_full_complex)
    mag_under = np.abs(image_under_complex)

    # Slice-wise NMSE
    nmses = [nmse(mag_gt[i], mag_under[i]) for i in range(mag_gt.shape[0])]
    mean_nmse = np.mean(nmses)

    nmse_all_volumes.append(mean_nmse)
    #print(f"{os.path.basename(file)}: NMSE = {mean_nmse:.6f}")

# Overall statistics
avg_nmse = np.mean(nmse_all_volumes)
std_nmse = np.std(nmse_all_volumes)

print("\n" + "=" * 40)
print(f"Overall Average NMSE: {avg_nmse:.6f} ± {std_nmse:.6f}")
print("=" * 40)



Overall Average NMSE: 0.055463 ± 0.025401


In [27]:
import numpy as np
import h5py
import os
import math
import matplotlib.pyplot as plt  # <-- Required for plotting
from typing import Optional
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from tqdm import tqdm
def psnr(
    gt: np.ndarray, pred: np.ndarray, maxval: Optional[float] = None
) -> np.ndarray:
    """Compute Peak Signal to Noise Ratio metric (PSNR)"""
    if maxval is None:
        maxval = gt.max()
    return peak_signal_noise_ratio(gt, pred, data_range=maxval)


# Your preprocessed folder
save_dir = r'D:\fastmri_singlecoil_FSSCAN\val_norm'
files = sorted([os.path.join(save_dir, f) for f in os.listdir(save_dir) if f.endswith('.h5')])
psnr_all_volumes = []

for file in files:
    with h5py.File(file, 'r') as f:
        image_full = f['image_full'][:]   # shape: [slices, H, W, 2]
        image_under = f['image_under'][:]
        max_val=f['max_val_full_image'][:]

    # Convert to complex
    #image_full_complex = image_full[..., 0] + 1j * image_full[..., 1]
    #image_under_complex = image_under[..., 0] + 1j * image_under[..., 1]

    # Convert to magnitude
    #mag_gt = np.abs(image_full_complex)
    #mag_under = np.abs(image_under_complex)
    image_full=image_full*max_val
    image_under=image_under*max_val
    
    psnrs = psnr(image_full,image_under,max_val)
    #mean_psnr = np.mean(psnrs)

    psnr_all_volumes.append(psnrs)
    #print(f"{os.path.basename(file)}: PSNR = {mean_psnr:.2f} dB")
# Overall average and standard deviation
overall_avg_psnr = np.mean(psnr_all_volumes)
overall_std_psnr = np.std(psnr_all_volumes)

print("\n" + "="*40)
print(f"Overall Average PSNR: {overall_avg_psnr:.6f} ± {overall_std_psnr:.6f} dB")
print("="*40)


Overall Average PSNR: 30.852452 ± 2.708017 dB


In [30]:
import os
import h5py
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
class SSIMMetric(nn.Module):
    def __init__(self, win_size: int = 7, k1: float = 0.01, k2: float = 0.03):
        super().__init__()
        self.win_size = win_size
        self.k1, self.k2 = k1, k2
        self.register_buffer("w", torch.ones(1, 1, win_size, win_size) / win_size**2)
        NP = win_size**2
        self.cov_norm = NP / (NP - 1)

    def forward(self, X: torch.Tensor, Y: torch.Tensor, data_range: torch.Tensor, reduced: bool = True):
        data_range = data_range[:, None, None, None]
        C1 = (self.k1 * data_range) ** 2
        C2 = (self.k2 * data_range) ** 2
        ux = F.conv2d(X, self.w)
        uy = F.conv2d(Y, self.w)
        uxx = F.conv2d(X * X, self.w)
        uyy = F.conv2d(Y * Y, self.w)
        uxy = F.conv2d(X * Y, self.w)
        vx = self.cov_norm * (uxx - ux * ux)
        vy = self.cov_norm * (uyy - uy * uy)
        vxy = self.cov_norm * (uxy - ux * uy)
        A1, A2, B1, B2 = (
            2 * ux * uy + C1,
            2 * vxy + C2,
            ux**2 + uy**2 + C1,
            vx + vy + C2,
        )
        D = B1 * B2
        S = (A1 * A2) / D
        return S.mean() if reduced else S
import os
import h5py
import numpy as np
from tqdm import tqdm
import torch
#from ssim_metric import SSIMMetric  # Import your SSIMMetric class

# Path to your preprocessed HDF5 validation volumes
save_dir = r'D:\fastmri_singlecoil_FSSCAN\val_norm'
files = sorted([os.path.join(save_dir, f) for f in os.listdir(save_dir) if f.endswith('.h5')])

ssim_all_volumes = []

# Initialize SSIM metric on CPU
ssim_metric = SSIMMetric(win_size=7, k1=0.01, k2=0.03)

for file in tqdm(files, desc="Computing SSIM over volumes"):
    with h5py.File(file, 'r') as f:
        image_full = f['image_full'][:]       # shape: [slices, H, W, 2]
        image_under = f['image_under'][:]     # same shape
        max_val=f['max_val_full_image'][:]


    image_full=image_full*max_val
    image_under=image_under*max_val
    # Convert complex to magnitude
    image_full_complex = image_full[..., 0] + 1j * image_full[..., 1]
    image_under_complex = image_under[..., 0] + 1j * image_under[..., 1]

    image_full_mag = np.abs(image_full_complex)
    image_under_mag = np.abs(image_under_complex)

    max_val = np.max(image_full_mag)

    ssim_slices = []
    for i in range(image_full_mag.shape[0]):
        gt = image_full_mag[i]
        pred = image_under_mag[i]

        # Convert to torch tensors with shape [1, 1, H, W]
        gt_tensor = torch.from_numpy(gt).unsqueeze(0).unsqueeze(0).float()
        pred_tensor = torch.from_numpy(pred).unsqueeze(0).unsqueeze(0).float()
        max_val_tensor = torch.tensor([max_val], dtype=torch.float32)

        ssim = ssim_metric(pred_tensor, gt_tensor, data_range=max_val_tensor, reduced=True)
        ssim_slices.append(ssim.item())

    ssim_volume = np.mean(ssim_slices)
    ssim_all_volumes.append(ssim_volume)

# Overall statistics
avg_ssim = np.mean(ssim_all_volumes)
std_ssim = np.std(ssim_all_volumes)

print("\n" + "=" * 40)
print(f"Overall Average SSIM: {avg_ssim:.6f} ± {std_ssim:.6f}")
print("=" * 40)


Computing SSIM over volumes: 100%|███████████████████████████████████████████████████| 199/199 [00:54<00:00,  3.68it/s]


Overall Average SSIM: 0.718261 ± 0.084263


In [28]:
import os
import numpy as np
import h5py
import glob
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import tensorflow.keras.backend as K
# Path to validation folder
# val_folder = "F:/denoised_preprocessed_h5_val"

# val_folder = r"E:\fastmri\val_norm"
# val_folder = r"D:\val_norm"

# val_folder = r"G:\val_norm\val_norm"
# files = sorted([os.path.join(val_folder, f) for f in os.listdir(val_folder) if f.endswith(".h5")])
kspace_files_list_val = sorted(glob.glob(os.path.join(val_folder, "*.h5")))
file_paths = kspace_files_list_val


# ----------------------
# HELPERS
# ----------------------
def to_complex(x):
    return x[..., 0] + 1j * x[..., 1]
def nmse(y_true, y_pred):
    return K.mean(K.square(y_true - y_pred)) / K.mean(K.square(y_true))
def nmse1(gt, pred):
    return np.linalg.norm(gt - pred) ** 2 / (np.linalg.norm(gt) ** 2 + 1e-10)

def compute_ssim(gt, pred, max_val):
    return structural_similarity(
        gt, pred,
        data_range=max_val,
        win_size=9,
        gaussian_weights=False,
        use_sample_covariance=False,
        K1=0.01,
        K2=0.03
    )

# ----------------------
# STORAGE
# ----------------------
ssim_list = []
psnr_list = []
nmse_list = []

# ----------------------
# PROCESSING
# ----------------------
for file in tqdm(file_paths, desc="Processing volumes"):
    with h5py.File(file, 'r') as f:
        image_full = f["image_full"][:]       # (slices, H, W, 2)
        image_under = f["image_under"][:]     # (slices, H, W, 2)
        max_val = float(f["max_val_full_image"][0])

#     mask_batch = np.tile(mask, (image_under.shape[0], 1, 1, 1)) 
    # Get model prediction (still in normalized form)
    # pred = model.predict([image_under,mask_batch,image_under], verbose=0)  # shape (slices, H, W, 2)
    #pred = model.predict(image_under, verbose=0)  # shape (slices, H, W, 2)
    
    #image_full *= max_val
    
    #image_under *= max_val  # Scale predicted output to original intensity range

    # Convert to complex and get magnitude
    gt_mag = np.abs(to_complex(image_full))
    pred_mag = np.abs(to_complex(image_under))

    # Volume-wise PSNR and NMSE
    psnr_val = peak_signal_noise_ratio(image_full, image_under, data_range=1.0)
    nmse_val = nmse(gt_mag.flatten(), pred_mag.flatten())

    psnr_list.append(psnr_val)
    nmse_list.append(nmse_val)

    # Slice-wise SSIM
    for i in range(gt_mag.shape[0]):
        ssim_val = compute_ssim(gt_mag[i], pred_mag[i], 1.0)
        ssim_list.append(ssim_val)

# ----------------------
# REPORT
# ----------------------
print("\n" + "=" * 40)
print(f"PSNR (Mag, volume): {np.mean(psnr_list):.2f} ± {np.std(psnr_list):.2f} dB")
print(f"NMSE (Mag, volume): {np.mean(nmse_list):.6f} ± {np.std(nmse_list):.6f}")
print(f"SSIM (Mag, slice):  {np.mean(ssim_list):.4f} ± {np.std(ssim_list):.4f}")

print("=" * 40)


Processing volumes: 100%|████████████████████████████████████████████████████████████| 199/199 [00:58<00:00,  3.38it/s]


PSNR (Mag, volume): 30.85 ± 2.71 dB
NMSE (Mag, volume): 0.038766 ± 0.017024
SSIM (Mag, slice):  0.7265 ± 0.0854


In [ ]:
import os
import h5py
import numpy as np
from tqdm import tqdm
from skimage.metrics import structural_similarity as compare_ssim

# Path to your preprocessed HDF5 validation volumes
save_dir = r'D:\fastmri\preprocessed_h5_val'
files = sorted([os.path.join(save_dir, f) for f in os.listdir(save_dir) if f.endswith('.h5')])

ssim_all_volumes = []

for file in tqdm(files, desc="Computing SSIM over volumes"):
    with h5py.File(file, 'r') as f:
        image_full = f['image_full'][:]       # shape: [slices, H, W, 2]
        image_under = f['image_under'][:]     # same shape
        #max_val = f['max_val_full'][()]       # scalar

    # Convert complex to magnitude
    image_full_complex = image_full[..., 0] + 1j * image_full[..., 1]
    image_under_complex = image_under[..., 0] + 1j * image_under[..., 1]
    
    image_full_mag = np.abs(image_full_complex)
    image_under_mag = np.abs(image_under_complex)
    max_val=np.max(image_full_mag)

    ssim_slices = []
    for i in range(image_full_mag.shape[0]):
        gt_mag = image_full_mag[i]
        pred_mag = image_under_mag[i]

        ssim_slice= compare_ssim(
            gt_mag, pred_mag,
            data_range=max_val,
            win_size=9,
            gaussian_weights=False,
            use_sample_covariance=False,      # <-- now correct per paper
            K1=0.01,
            K2=0.03,
            full=False
        )
        ssim_slices.append(ssim_slice)

    ssim_volume = np.mean(ssim_slices)
    ssim_all_volumes.append(ssim_volume)

# Overall statistics
avg_ssim = np.mean(ssim_all_volumes)
std_ssim = np.std(ssim_all_volumes)

print("\n" + "=" * 40)
print(f"Overall Average SSIM: {avg_ssim:.6f} ± {std_ssim:.6f}")
print("=" * 40)


In [8]:
#%run "C:\\Users\\Research\\aman fastmri\\cx-Copy1.ipynb"
%run "./model_unet.ipynb"

In [10]:
from tensorflow.keras.optimizers import RMSprop

In [14]:
from sklearn.metrics import mean_squared_error
import os

class VolumeWiseNMSE(tf.keras.callbacks.Callback):
    def __init__(self, val_files, save_best_path):
        self.val_files = val_files
        self.best_nmse = float('inf')
        self.save_best_path = save_best_path

    def on_epoch_end(self, epoch, logs=None):
        total_nmse = 0
        num_volumes = 0

        for file_path in self.val_files:
            with h5py.File(file_path, 'r') as f:
                image_under = f['image_under'][:]  # [slices, H, W, 2]
                gt = f['image_full'][:]

            pred = []
            for i in range(image_under.shape[0]):
                input_slice = np.expand_dims(image_under[i], axis=0)
                pred_slice = self.model.predict(input_slice, verbose=0)
                pred.append(pred_slice[0])
            pred = np.stack(pred)

            nmse = np.sum((np.abs(gt - pred) ** 2)) / np.sum((np.abs(gt) ** 2))
            total_nmse += nmse
            num_volumes += 1

        avg_nmse = total_nmse / num_volumes
        print(f"\nEpoch {epoch + 1} - Avg NMSE: {avg_nmse:.6f}")

        if avg_nmse < self.best_nmse:
            print(f"New best model found! Saving model with NMSE {avg_nmse:.6f}")
            self.best_nmse = avg_nmse
            self.model.save(self.save_best_path)


In [16]:

H, W = 320, 320
model = unet(H=H, W=W, channels=2, kshape=(3, 3))
opt = RMSprop(learning_rate=0.001)

model_name = "./SavedModels/best_model_nmse_full_volume.h5"
model.compile(loss='mse', optimizer=opt, metrics=[nmse])

# Load weights if available
if os.path.isfile(model_name):
    model.load_weights(model_name)
    print("Weights loaded")

nmse_callback = VolumeWiseNMSE(kspace_files_list_val, save_best_path=model_name)

class StepDecayLR(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        if epoch == 40:
            old_lr = float(tf.keras.backend.get_value(self.model.optimizer.lr))
            new_lr = old_lr * 0.1
            tf.keras.backend.set_value(self.model.optimizer.lr, new_lr)
            print(f"\nEpoch {epoch + 1}: Reducing learning rate from {old_lr} to {new_lr}")

model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=50,
    callbacks=[nmse_callback, StepDecayLR()]
)


Epoch 1/50


NotImplementedError: in user code:

    File "C:\Users\DU\anaconda3\envs\WNet\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\DU\AppData\Local\Temp\ipykernel_13632\1966064014.py", line 27, in nmse  *
        return np.linalg.norm(gt - pred) ** 2 / (np.linalg.norm(gt) ** 2 + 1e-10)
    File "C:\Users\DU\anaconda3\envs\WNet\lib\site-packages\numpy\linalg\linalg.py", line 2534, in norm  **
        x = asarray(x)

    NotImplementedError: Cannot convert a symbolic tf.Tensor (sub:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported.
